In [1]:
import os, json
import numpy as np
import pandas as pd
from dLoader import DataLoader

# Get Tickers from Database

In [41]:
directory = os.path.join(os.getcwd(), 'Database')
# Search Param
min_dollar = 0
max_dollar = 5
min_length = 250 * 6
today = pd.Timestamp.now()
# Placeholder
tickers = []
files = os.listdir(directory)
if 'keeper.json' in files:
    path = os.path.join(directory, 'keeper.json')
    db = pd.read_json(path, orient='index')
    # Get pricing
    mask = (db['Latest Price'].between(min_dollar, max_dollar)) & (db['Length in Database'] > min_length)
    ndb = db.loc[mask].copy()
    need_update = (today.day - pd.to_datetime(ndb['Last Updated']).dt.day) > 5
    if need_update.sum() > 0:
        jFile = json.load(open(path, 'r'))
        for ticker in ndb.loc[need_update].index.to_list():
            DataLoader(ticker).update_database()
            jFile[ticker]['Last Updated'] = today.strftime('%Y-%m-%d')
        # Write json file
        json.dump(jFile, open(path, 'w'), indent=4)
    tickers = db.loc[mask].index.to_list()
else:
    jFile = {}
    for file in files:
        name, ext = file.split('.')
        if ext == 'csv':
            df = DataLoader(name).data
            latest_price = df['Close'].iloc[-1]
            jFile[name] = {'Latest Price': latest_price,
                           'Length in Database': len(df),
                           'Last Updated': df.index[-1].strftime('%Y-%m-%d')}
            if len(df) > min_length and (min_dollar < latest_price < max_dollar):
                tickers.append(name)
    # Write json file
    with open(os.path.join(directory, 'keeper.json'), 'w') as outfile:
        json.dump(jFile, outfile, indent=4)

print('Total tickers met parameters in Database: {}'.format(len(tickers)))

Total tickers met parameters in Database: 145


In [42]:
def sim(data, sell_limit=.03, buy_limit=.03, max_hold=10, capital=1000, max_shares=1000):
    data['Previous_Close'] = data['Close'].shift(1)
    base_capital = capital
    shares = 0
    holding = False
    buy_at = 0
    held = 0
    is_selling = False
    for d, prices in data.iterrows():
        if pd.isna(prices.Previous_Close):
            continue
        if not holding:
            check_low = prices.Low / prices.Previous_Close - 1
            if check_low < -buy_limit:
                buy_at = prices.Low * (1 + buy_limit)
                shares = capital // buy_at
                shares = shares if shares <= max_shares else max_shares
                capital -= buy_at * shares
                holding = True
                is_selling = False
        else:
            held += 1
            check_open = prices.Open / buy_at - 1
            check_high = prices.High / buy_at - 1
            if check_open > sell_limit:
                sell_price = prices.Open
                is_selling = True
            elif check_high > sell_limit:
                sell_price = buy_at * (1 + sell_limit)
                is_selling = True
            elif held > max_hold:
                sell_price = prices.Close
                is_selling = True
            
            if is_selling:
                capital += sell_price * shares
                shares = 0
                buy_at = 0
                holding = False
                held = 0
                is_selling = False
    if holding:
        capital += data.iloc[-1]['Close'] * shares

    return capital / base_capital - 1

In [43]:
start_dates = pd.date_range('2018-01-01', '2020-12-31', freq='QS')
end_dates = pd.date_range('2018-01-01', '2020-12-31', freq='Q')
dic = {}
for ticker in tickers:
    stat = []
    try:
        for start, end in zip(start_dates, end_dates):
            data = DataLoader(ticker).get_data(start.strftime('%Y-%m-%d'), 
                                            end.strftime('%Y-%m-%d'))
            stat.append(sim(data))
            if len(stat) > 4 and np.mean(stat) < .2:
                break
        avg_quarterly_status = np.mean(stat)
    except Exception as e:
        continue
    dic[ticker] = {'Cap Gain': avg_quarterly_status}

In [44]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [45]:
mask = df['Cap Gain'].between(.6, 10)
backtest_tickers = df[mask].index.to_list()
backtest_tickers

['ATOS', 'BTX', 'GBOX', 'HGEN', 'UAVS']

In [52]:
base_capital = 1000
start_dates = pd.date_range('2021-01-01', '2021-12-31', freq='QS')
end_dates = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
total_gain = {ticker: {'Quarterly Cap Gain': 0,
                       'Cap Gain': 0,
                       'Base Cap': 0,
                       'End Dollar': 0,
                       'Stop': 0} 
              for ticker in backtest_tickers}
for ticker in backtest_tickers:
    sect_cap = base_capital / len(backtest_tickers)
    total_gain[ticker]['Base Cap'] = sect_cap
    stat = []
    try:
        for start, end in zip(start_dates, end_dates):
            data = DataLoader(ticker).get_data(start.strftime('%Y-%m-%d'), 
                                            end.strftime('%Y-%m-%d'))
            score = sim(data, capital=sect_cap, max_shares=10000)
            sect_cap = sect_cap * (1 + score)
            stat.append(score)
            if score < .6:
                break
        total_gain[ticker]['Quarterly Cap Gain'] = np.mean(stat)
        total_gain[ticker]['End Dollar'] = sect_cap
        total_gain[ticker]['Cap Gain'] = sect_cap - total_gain[ticker]['Base Cap']
        total_gain[ticker]['Stop'] = end
    except Exception as e:
        continue

In [53]:
df = pd.DataFrame.from_dict(total_gain, orient='index')
df

,Quarterly Cap Gain,Cap Gain,Base Cap,End Dollar,Stop
ATOS,0.630849,438.819215,200.0,638.819215,2021-09-30
BTX,1.090876,802.618225,200.0,1002.618225,2021-09-30
GBOX,0.804270,418.740421,200.0,618.740421,2021-06-30
HGEN,0.434452,86.890389,200.0,286.890389,2021-03-31
UAVS,0.204841,40.968103,200.0,240.968103,2021-03-31


In [54]:
df['Cap Gain'].sum(), df['End Dollar'].sum()

(1788.036353508001, 2788.0363535080014)